In [2]:
import pandas as pd
import numpy as np

import datetime
from collections import Counter


In [3]:
hvc_hvvisitresult = pd.read_csv(".\HVC_HVVISITRESULT.csv", sep=";")
hvc_so0 = pd.read_csv(".\HVC_SO0.csv", sep=";")
hvc_am0 = pd.read_csv(".\HVC_AM0.csv", sep=";")
hvc_ar0 = pd.read_csv(".\HVC_AR0.csv", sep=";")
hvc_hvvisitresultdetails = pd.read_csv(".\HVC_VISITRESULTDETAILS.csv", sep=";")
hvc_customerreviews = pd.read_csv(".\HVC_CUSTOMER_REVIEWS.csv", sep=";")
hvc_depot = pd.read_csv(".\HVC_DEPOT.csv", sep=";")
hvc_hvdayofweek = pd.read_csv(".\HVC_HVDAYOFWEEK.csv", sep=";")
hvc_hvposition = pd.read_csv(".\HVC_HVPOSITION.csv", sep=";")
hvc_hvroutetemplate = pd.read_csv(".\HVC_HVROUTETEMPLATE.csv", sep=";")
hvc_hvvisitoutcome = pd.read_csv(".\HVC_HVVISITOUTCOME.csv", sep=";")
hvc_hvvisitresult = pd.read_csv(".\HVC_HVVISITRESULT.csv", sep=";")

# Explore the data

## Products

In [12]:
hvc_ar0.head(10)

,SO0_NRID,HVROUTETEMPLATE_NRID,CUST_TYPE,POSTCODE,LANGUAGE,SEASON_TYPE
0,721104.0,219020601.0,Private,3930,nl-BE,All time
1,721110.0,219020601.0,Private,3930,nl-BE,All time
2,721121.0,219020601.0,Private,3930,nl-BE,All time
3,721126.0,219020601.0,Private,3930,nl-BE,All time
4,721131.0,219020601.0,Private,3930,nl-BE,All time
5,721136.0,219020601.0,Private,3930,nl-BE,All time
6,721156.0,219020601.0,Private,3930,nl-BE,All time
7,721168.0,219020601.0,Private,3930,nl-BE,All time
8,721173.0,219020601.0,Private,3930,nl-BE,All time
9,721200.0,219021241.0,Private,3910,nl-BE,All time


In [13]:
len(hvc_ar0["AR0_NRID"].unique())

5709

In [14]:
hvc_ar0["FAMILY"].value_counts(normalize=True)

,LANGUAGE,SEASON_TYPE
count,5709,5708
unique,2,4
top,nl-BE,All time
freq,5703,5554


In [15]:
hvc_ar0["PRICE"].describe()

nl-BE    5703
fr-BE       6
Name: LANGUAGE, dtype: int64

In [ ]:
hvc_ar0[hvc_ar0["PRICE"] < 0]

In [ ]:
#What are the most frequently bought products? 
#Which products render the most revenue? 
#Which products are bought the most in the region of Brussels, Antwerp, ...? 
#Which products render the most revenue in the region of... ?
#What are the total sales generated for each product family? 
#Does the weather / seasonal changes have an effect on the total revenue?

#What are the routes of the employees?
#What is the turnover for each employee?

##### Tot hier gedaan ####

#Are product purchases correlated? Are some products often purchased together? 


#Analyze the other datasets as well (employees, customers, products, routes, ...)
#How many customers visited each employee?
#Are there customers that stopped buying products?
#Who are the best customers in terms of CLV?
#Who are the top performing employees?
#What are the best selling products?
#What are the worst selling products?

# Eigen code: antwoorden voor 'customers'

In [12]:
def most_frequent(List):
    occurence_count = Counter(List)
    return occurence_count.most_common(1)[0][0]
   
#Most FREQUENTLY bought item (not most bought item) --> nog een manier vinden om de eerste 10 ofzo te vinden
#print(most_frequent(hvc_hvvisitresultdetails["AR0_NRID"]))

#Productid with their total sold quantity
sold_quantity = hvc_hvvisitresultdetails.groupby("AR0_NRID")["QUANTITY"].sum("QUANTITY").to_frame()
sold_quantity.columns = ["TOTAL_QUANTITY"]
sold_qt_sorted = sold_quantity.sort_values(by=["TOTAL_QUANTITY"], ascending=False)
sold_qt_sorted.head(5)

#Products and their total revenue
results_products = sold_quantity.merge(hvc_ar0, on="AR0_NRID", how="inner")
results_products["TOTAL_REVENUE"] = results_products["TOTAL_QUANTITY"] * results_products["PRICE"]
results_products = results_products.sort_values(by=["TOTAL_REVENUE"], ascending=False)
results_products.head(5)


#Most bought product for a depot
depot = "Antwerpen"
temporary_df = hvc_depot.merge(hvc_so0, on="HVROUTETEMPLATE_NRID", how="inner") \
        .merge(hvc_hvvisitresult, on="SO0_NRID", how="inner") \
        .merge(hvc_hvvisitresultdetails, on="HVVISITRESULT_NRID", how="inner") \
        .merge(hvc_ar0, on = "AR0_NRID", how = "inner")

products_per_region = temporary_df[["DEPOT", "AR0_NRID", "QUANTITY", "PRICE"]]
total_products_per_region = products_per_region.groupby(["DEPOT", "AR0_NRID", "QUANTITY", "PRICE"], as_index = False)["QUANTITY"].sum("QUANTITY")
total_products_per_region = total_products_per_region.sort_values(by=["QUANTITY"], ascending=False)

#Most revenue per product for a certain depot
total_products_per_region["TOTAL_REVENUE"] = total_products_per_region["QUANTITY"] * total_products_per_region["PRICE"]
total_products_per_region = total_products_per_region.sort_values(by=["TOTAL_REVENUE"], ascending=False)
total_products_for_a_region = total_products_per_region[total_products_per_region['DEPOT']==depot]
total_products_for_a_region.head(5)

#Total quantity sold for a productfamily
sold_quantity = hvc_hvvisitresultdetails.groupby("AR0_NRID")["QUANTITY"].sum("QUANTITY")
sold_quantity_df = sold_quantity.to_frame()
sold_quantity_df.columns = ["TOTAL_QUANTITY"]
sold_qt_sorted = sold_quantity_df.sort_values(by=["TOTAL_QUANTITY"], ascending=False)
sold_qt_sorted.head(5)

#Productfamilies and their total revenue
results_products.groupby(["FAMILY"])["FAMILY", "TOTAL_REVENUE"].sum("TOTAL_REVENUE").sort_values(by=["TOTAL_REVENUE"], ascending=False)

## Seasonal effects
winter_customers = hvc_so0[hvc_so0["SEASON_TYPE"] == "Winter"]
summer_customers = hvc_so0[hvc_so0["SEASON_TYPE"] == "Summer"]
alltime_customers = hvc_so0[hvc_so0["SEASON_TYPE"] == "All time"]
christmas_customers = hvc_so0[hvc_so0["SEASON_TYPE"] == "Christmas"]

seasonal_customers = alltime_customers

#total amount per season
seasons_totalamount = hvc_so0.merge(hvc_hvvisitresult, on="SO0_NRID", how="inner") \
                    .merge(hvc_hvvisitresultdetails, on="HVVISITRESULT_NRID", how="inner") \
                    .merge(hvc_ar0, on="AR0_NRID", how="inner")\
                    .groupby(["SEASON_TYPE"])["SEASON_TYPE", "AMOUNT"].sum("AMOUNT")
                    
#average amount for 1 season over the customers
def seasonal_average(seasonal_customers):
    season_averageamount = seasonal_customers.merge(hvc_hvvisitresult, on="SO0_NRID", how="inner") \
                    .merge(hvc_hvvisitresultdetails, on="HVVISITRESULT_NRID", how="inner") \
                    .merge(hvc_ar0, on="AR0_NRID", how="inner") \
                    .groupby(["SEASON_TYPE"])["SEASON_TYPE", "AMOUNT"].sum("AMOUNT")
    season_averageamount["AMOUNT"] = season_averageamount["AMOUNT"] / len(seasonal_customers)
    return season_averageamount

#alle seizoenen samenzetten --> beetje messy oplossing wel, maar t werkt
averageamount_seasons = seasonal_average(alltime_customers).append(seasonal_average(winter_customers)).append(seasonal_average(summer_customers)).append(seasonal_average(christmas_customers))
averageamount_seasons



23678


<ipython-input-12-acac9c7fe58d>:46: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  results_products.groupby(["FAMILY"])["FAMILY", "TOTAL_REVENUE"].sum("TOTAL_REVENUE").sort_values(by=["TOTAL_REVENUE"], ascending=False)
<ipython-input-12-acac9c7fe58d>:57: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  seasons_totalamount = hvc_so0.merge(hvc_hvvisitresult, on="SO0_NRID", how="inner") \
<ipython-input-12-acac9c7fe58d>:64: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  season_averageamount = seasonal_customers.merge(hvc_hvvisitresult, on="SO0_NRID", how="inner") \


,AMOUNT
SEASON_TYPE,
All time,505.616520
Winter,53.800000
Summer,359.117333
Christmas,420.200000


In [36]:
# define a function that accepts a customer id, the transactions data, transactions details data and product data
def get_products(cust_id, transactions, transaction_details, products):
    

    # join transactions with transaction details with products
    transactions_products = transactions.merge(transaction_details, on="HVVISITRESULT_NRID", how="inner") \
                                        .merge(products, on="AR0_NRID", how="inner")
    
    # get transactions of customer 
    transactions_products_cust = transactions_products[transactions_products["SO0_NRID"] == cust_id]
    
    # only get transactions of succesfull visits
    transactions_products_cust = transactions_products_cust[transactions_products_cust["HVOUTCOME_NRID"] == 2]
    
    # get total amount of products bought by the customer
    total_products = transactions_products_cust["QUANTITY"].sum()
    
    # get total number of unique products bought by the customer
    total_unique_products = len(transactions_products_cust["DESCRIPTION"].unique())
    
    # return
    return(total_products, total_unique_products)

# Eigen code: antwoorden voor 'Employees'

In [174]:
#Employees and their routes
employee_routes = hvc_so0.merge(hvc_hvvisitresult, on="SO0_NRID", how="inner")
employee_routes = employee_routes[["AM0_NRID","HVROUTETEMPLATE_NRID"]].sort_values(by=["AM0_NRID"]).drop_duplicates()
employee_routes

#Employees and their total results
hvc_hvvisitresult.groupby("AM0_NRID")["AM0_NRID", "AMOUNT"].sum("AMOUNT").sort_values(by = ["AMOUNT"], ascending = False)


<ipython-input-174-0c41757c9f17>:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  hvc_hvvisitresult.groupby("AM0_NRID")["AM0_NRID", "AMOUNT"].sum("AMOUNT").sort_values(by = ["AMOUNT"], ascending = False)


,AM0_NRID,AMOUNT
AM0_NRID,,
18934048107254,1029860744650210176,286009.10
19946584592834,1060559902802432512,265728.55
24034952445848,1057537907616615808,186259.10
23186288605042,527047526281430016,94536.60
25146868694050,476960658520250048,54292.00
20126680425436,181381643994058112,47835.50
24705284396450,404672558413970560,46556.10
23845760556442,149202923801672480,20819.00
24871084536436,112093978005704816,16934.75


# Vanaf hier beetje eigen vragen opgelost

In [203]:
# Which customers buy most frequently?
# Which customers buy for the highest amounts?
# How many successful visits per month / day?
# Is the business growing (eg higher total amounts or higher total quantity sold over the years/months)?
# Focus on NL or FR speaking customers?
# Which products do frequent customers buy? OK
# Which products were only bought a few times, then never again?
# What % of our routes have positive outcomes?
# Which routes have the most positive outcomes?
# Which routes are unnecessary?
# Which customer types should we focus on?
# Customer reviews: 
# for happy & angry customers: what products have they bought
# and which employees visitited them
# Which depots are best selling? Which depots can go / where do we need extra depots
# Which depots have most customers?
# Test new products on best buying customers, or do they always buy the same?

In [273]:
# Which customers have highest succes rate during the visits?
succesful_visits = hvc_hvvisitresult[hvc_hvvisitresult["HVOUTCOME_NRID"] == 2].groupby("SO0_NRID")["SO0_NRID"].count().to_frame()
succesful_visits.columns = ["TOTAL_SUCCESSFUL_VISITS"]
succesful_visits.sort_values(by = "TOTAL_SUCCESSFUL_VISITS", ascending = False)

unsuccesful_visits = hvc_hvvisitresult[hvc_hvvisitresult["HVOUTCOME_NRID"] == 0].groupby("SO0_NRID")["SO0_NRID"].count().to_frame()
unsuccesful_visits.columns = ["TOTAL_UNSUCCESSFUL_VISITS"]
unsuccesful_visits.sort_values(by = "TOTAL_UNSUCCESSFUL_VISITS", ascending = False)

visit_outcomes = succesful_visits.merge(unsuccesful_visits, on="SO0_NRID", how="inner")
visit_outcomes["TOTAL_VISITS_WHEN_HOME"] = visit_outcomes["TOTAL_SUCCESSFUL_VISITS"] + visit_outcomes["TOTAL_UNSUCCESSFUL_VISITS"]
visit_outcomes["%UNSUCCESSFUL"] = visit_outcomes["TOTAL_UNSUCCESSFUL_VISITS"] / visit_outcomes["TOTAL_VISITS_WHEN_HOME"]
visit_outcomes["%SUCCESSFUL"] = visit_outcomes["TOTAL_SUCCESSFUL_VISITS"] / visit_outcomes["TOTAL_VISITS_WHEN_HOME"]

visit_outcomes = visit_outcomes.sort_values(by="%SUCCESSFUL", ascending = False)
visit_outcomes

,TOTAL_SUCCESSFUL_VISITS,TOTAL_UNSUCCESSFUL_VISITS,TOTAL_VISITS_WHEN_HOME,%UNSUCCESSFUL,%SUCCESSFUL
SO0_NRID,,,,,
1246638,53,1,54,0.018519,0.981481
1249611,52,1,53,0.018868,0.981132
1401701,49,1,50,0.020000,0.980000
21636848473050,47,1,48,0.020833,0.979167
1252217,47,1,48,0.020833,0.979167
...,...,...,...,...,...
728405,1,41,42,0.976190,0.023810
21251384405050,1,41,42,0.976190,0.023810
1253835,2,88,90,0.977778,0.022222


In [257]:
#Which products are bought by customers with highest succes rate during visits?
#vraag forum over groupby
top_successful_customers = visit_outcomes[visit_outcomes["%SUCCESSFUL"] > 0.75]
products_successful_customers = top_successful_customers.merge(hvc_hvvisitresult, on="SO0_NRID").merge(hvc_hvvisitresultdetails, on="HVVISITRESULT_NRID").groupby("AR0_NRID").sum("QUANTITY")
#products_successful_customers = products_successful_customers["AR0_NRID", "AMOUNT"].to_frame().drop_duplicates()
products_successful_customers = products_successful_customers[["AMOUNT", "QUANTITY"]].sort_values(by="QUANTITY", ascending = False)
products_successful_customers

,AMOUNT,QUANTITY
AR0_NRID,,
23678,40959.30,2834.0
23716,43145.35,2044.0
23685,19083.30,1963.0
23734,16219.80,920.0
23714,15981.90,793.0
...,...,...
23772,11.40,1.0
23703,15.70,1.0
23796,32.80,1.0


In [274]:
#Which products are bought by customers with lowest success rate?
#vraag forum over groupby
top_nonsuccessful_customers = visit_outcomes[visit_outcomes["%SUCCESSFUL"] < 0.25]
products_nonsuccessful_customers = top_nonsuccessful_customers.merge(hvc_hvvisitresult, on="SO0_NRID").merge(hvc_hvvisitresultdetails, on="HVVISITRESULT_NRID").groupby("AR0_NRID").sum("QUANTITY")
products_nonsuccessful_customers = products_nonsuccessful_customers[["AMOUNT", "QUANTITY"]].sort_values(by="QUANTITY", ascending = False)
products_nonsuccessful_customers
#products_nonsuccessful_customers[products_nonsuccessful_customers["AR0_NRID"] == 66303120]


,AMOUNT,QUANTITY
AR0_NRID,,
23678,9029.40,746.0
23716,14503.25,627.0
23685,3356.60,355.0
23734,4131.55,305.0
23699,5292.10,195.0
...,...,...
18832444153838,176.05,1.0
60673327,76.00,1.0
50649538,58.30,1.0


In [283]:
#Difference in products: products that nonsuccessful customers (<0.25 success rate) buy, that successful customers (success rate > 0.75) do not buy
top_successful_customers = visit_outcomes[visit_outcomes["%SUCCESSFUL"] > 0.75]
products_successful_customers = top_successful_customers.merge(hvc_hvvisitresult, on="SO0_NRID").merge(hvc_hvvisitresultdetails, on="HVVISITRESULT_NRID").groupby("AR0_NRID").sum("QUANTITY")
products_successful_customers = products_successful_customers[["AMOUNT", "QUANTITY"]].sort_values(by="QUANTITY", ascending = False)
nonsuccessful_products = products_nonsuccessful_customers.merge(products_successful_customers, on = "AR0_NRID", how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only']
nonsuccessful_products[["AMOUNT_x", "QUANTITY_x"]]

,AMOUNT_x,QUANTITY_x
AR0_NRID,,
23764,225.00,18.0
17925360348038,1109.60,7.0
21276088156846,1342.80,7.0
17545160268040,1257.70,7.0
47703913,54.80,6.0
17422560668040,1126.60,5.0
57367272,102.60,5.0
20413488326838,112.40,4.0
17794188535052,711.00,4.0


In [311]:
#Products that were not bought in the last year 2020
products_bought_2020 = hvc_ar0.merge(hvc_hvvisitresultdetails, on="AR0_NRID", how="inner")\
                              .merge(hvc_hvvisitresult, on="HVVISITRESULT_NRID", how="inner")
products_bought_2020['YEAR'] = pd.DatetimeIndex(products_bought_2020['DATEONLY']).year
products_bought_2020 = products_bought_2020[products_bought_2020["YEAR"] == 2020][["AR0_NRID"]].drop_duplicates()
all_products = hvc_ar0[["AR0_NRID"]]
#all_products
#products_bought_2020
products_not_bought_in_2020 = pd.concat([all_products,products_bought_2020]).drop_duplicates(keep=False)

42

In [1]:
# convert from string format to datetime format
def to_date(input):
    format = '%Y-%m-%d'
    date = datetime.strptime(input, format)
    return date.date()

#How many visits and their succes rate per day and per day of the week
succesful_visits = hvc_hvvisitresult[hvc_hvvisitresult["HVOUTCOME_NRID"] == 2]
unsuccesful_visits = hvc_hvvisitresult[hvc_hvvisitresult["HVOUTCOME_NRID"] == 0]
empty_visits = hvc_hvvisitresult[hvc_hvvisitresult["HVOUTCOME_NRID"] == 1]

succesful_visits_perday = succesful_visits.groupby("DATEONLY")["DATEONLY"].count().to_frame()
succesful_visits_perday.columns = ["#SUCCESS_VISITS"]

unsuccesful_visits_perday = unsuccesful_visits.groupby("DATEONLY")["DATEONLY"].count().to_frame()
unsuccesful_visits_perday.columns = ["#UNSUCCESS_VISITS"]

empty_visits_perday = empty_visits.groupby("DATEONLY")["DATEONLY"].count().to_frame()
empty_visits_perday.columns = ["#EMPTY_VISITS"]

visits_perday = succesful_visits_perday.merge(unsuccesful_visits_perday, on="DATEONLY", how="inner").merge(empty_visits_perday, on="DATEONLY", how="inner")
dates_list = [datetime.strptime(date, "%Y-%m-%d").date() for date in visits_perday.index.format()]
days_list = [date.strftime('%A') for date in dates_list]
visits_perday["WEEKDAY"] = days_list
visits_perday

visits_perdayofweek = visits_perday.groupby("WEEKDAY").sum()
visits_perdayofweek["TOTAL_VISITS"] = visits_perdayofweek["#SUCCESS_VISITS"] + visits_perdayofweek["#UNSUCCESS_VISITS"] + visits_perdayofweek["#EMPTY_VISITS"]
visits_perdayofweek["%SUCCESS"] = visits_perdayofweek["#SUCCESS_VISITS"] / visits_perdayofweek["TOTAL_VISITS"]
visits_perdayofweek["%UNSUCCESS"] = visits_perdayofweek["#UNSUCCESS_VISITS"] / visits_perdayofweek["TOTAL_VISITS"]
visits_perdayofweek["%EMPTY"] = visits_perdayofweek["#EMPTY_VISITS"] / visits_perdayofweek["TOTAL_VISITS"]
visits_perdayofweek

NameError: name 'hvc_hvvisitresult' is not defined

In [382]:
#What routes have highest success rates?
routes_success = hvc_so0.merge(hvc_hvvisitresult, on="SO0_NRID", how="inner")
routes_success = routes_success[routes_success["HVOUTCOME_NRID"] == 2]
routes_success.groupby("HVROUTETEMPLATE_NRID")